In [1]:
import os,sys,glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as st
import functions as fun
sourcedir = '/Volumes/My Passport/cmip5/cmip5'

In [2]:
basin = ['eais','wedd','amun','ross','apen']
imods = ['AISM_VUB','BISI_LBL','CISM_NCA','FETI_VUB','GRIS_LSC','IMAU_VUB','ISSM_JPL','ISSM_UCI','MALI_LAN','PISM_AWI','PISM_DMI','PISM_PIK','PISM_VUW','PS3D_PSU','SICO_UHO','UA_UNN']
bm = 11.5

In [3]:
def append_TH(T0,T1,T2,T3,T4,files,mask,vol):
    for fil in files:
        startyear = np.int(fil[-16:-12])
        if startyear>2100: continue
        print('Added TH',startyear)
        with xr.open_dataset(fil) as ds:
            th = ds['thetao'].values    
        tt = th[0::12,:,:]
        for f in range(1,12):
            ttnew = th[f::12,:,:]
            tt = tt[:ttnew.shape[0],:,:]+ttnew
        T0 = np.append(T0,np.nansum(tt*mask[0,:,:,:]*vol,axis=(1,2,3))/np.nansum(12.*mask[0,:,:,:]*vol))
        T1 = np.append(T1,np.nansum(tt*mask[1,:,:,:]*vol,axis=(1,2,3))/np.nansum(12.*mask[1,:,:,:]*vol))
        T2 = np.append(T2,np.nansum(tt*mask[2,:,:,:]*vol,axis=(1,2,3))/np.nansum(12.*mask[2,:,:,:]*vol))
        T3 = np.append(T3,np.nansum(tt*mask[3,:,:,:]*vol,axis=(1,2,3))/np.nansum(12.*mask[3,:,:,:]*vol))
        T4 = np.append(T4,np.nansum(tt*mask[4,:,:,:]*vol,axis=(1,2,3))/np.nansum(12.*mask[4,:,:,:]*vol))
    return T0,T1,T2,T3,T4    

In [4]:
def getgrid(model):
    #Get Antarctic mask
    with xr.open_dataset(f'../rawdata/Amaskdyn/{model}.nc') as ds:
        lon = ds['lon'].values
        lat = ds['lat'].values
        mask = ds['mask'].values
    return mask,lon,lat

def getvol(model):
    try:
        with xr.open_dataset(f'{sourcedir}/fx/volcello/{model}/r0i0p0/volcello_fx_{model}_historical_r0i0p0.nc') as ds:
            vol = ds['volcello'].values
    except:
        with xr.open_dataset(f'{sourcedir}/fx/areacello/{model}/r0i0p0/areacello_fx_{model}_historical_r0i0p0.nc') as ds:
            area = ds['areacello'].values
        files = glob.glob(f'{sourcedir}/{model}/r1i1p1/thetao_Omon_{model}_rcp85*')
        with xr.open_dataset(files[0]) as ds:
            lev = ds['lev_bnds'].values
        try:
            with xr.open_dataset(files[0]) as ds:
                dep = ds['depth'].values
            lev = lev[:,1]-lev[:,0]
            area2 = np.repeat(area[np.newaxis,:,:],len(lev),axis=0)
            lev2 = np.repeat(lev[:,np.newaxis],area.shape[0],axis=1)
            lev2 = np.repeat(lev2[:,:,np.newaxis],area.shape[1],axis=2)
            dep2 = np.repeat(dep[np.newaxis,:,:],len(lev),axis=0)
            vol = -dep2*area2*lev2
        except:
            lev = lev[:,1]-lev[:,0]
            area2 = np.repeat(area[np.newaxis,:,:],len(lev),axis=0)
            lev2 = np.repeat(lev[:,np.newaxis],area.shape[0],axis=1)
            lev2 = np.repeat(lev2[:,:,np.newaxis],area.shape[1],axis=2)
            vol = lev2*area2
    return vol

In [5]:
def gethist(model):
    #Get historical
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/thetao_Omon_{model}_historical*')
    Th0 = np.array([])
    Th1 = np.array([])
    Th2 = np.array([])
    Th3 = np.array([])
    Th4 = np.array([])
    Th0,Th1,Th2,Th3,Th4 = append_TH(Th0,Th1,Th2,Th3,Th4,files,mask,vol)
    print(model,'got hist')
    lastyear = np.int(files[-1][-9:-5])
    if lastyear>2005: 
        Th0 = Th0[:(2005-lastyear)]
        Th1 = Th1[:(2005-lastyear)]        
        Th2 = Th2[:(2005-lastyear)]
        Th3 = Th3[:(2005-lastyear)]
        Th4 = Th4[:(2005-lastyear)]
    firstyear = np.int(files[0][-16:-12])
    
    return Th0,Th1,Th2,Th3,Th4,firstyear

In [6]:
def getproj(model,scen,Th0,Th1,Th2,Th3,Th4,firstyear):
    
    #Get projected
    files = glob.glob(f'{sourcedir}/{model}/r1i1p1/thetao_Omon_{model}_{scen}*')
    T0,T1,T2,T3,T4 = append_TH(Th0,Th1,Th2,Th3,Th4,files,mask,vol)
    print(model,scen,'got proj')
    years = np.arange(firstyear,firstyear+len(T0))
    
    TH = np.array([T0,T1,T2,T3,T4])

    #Detrend
    for b in range(0,5):
        TH[b,:] = fun.detrend(TH[b,:],TH[b,:150],years)
        TH[b,:] = TH[b,:]-np.nanmean(TH[b,:150])
        
    #Extract 200 year time period 1900-2100
    TH = TH[:,np.logical_and(years>1900,years<2101)]
    years = years[np.logical_and(years>1900,years<2101)]
    
    return TH,years

In [7]:
def calcSLR(TH,years):
    SLRens = np.zeros((len(imods),TH.shape[0],TH.shape[1]))
    for i,imod in enumerate(imods):
        for b,bb in enumerate(range(1,6)):
            with open(f'../rawdata/Larmip2019-master/RFunctions/RF_{imod}_BM08_R{bb}.dat') as f:
                R = np.array([float(x) for x in f.readlines()])
            for t,tt in enumerate(years):
                if t==0: continue
                SLRens[i,b,t] = bm*100*np.sum(TH[b,:t][::-1]*R[:t])
            SLRens[i,b,:] = fun.detrend(SLRens[i,b,:],np.array([]),years)       
    SLRb = np.sum(SLRens,axis=0)/len(imods)
    return SLRb,years

In [8]:
def save_extra(TH,years,model,scen):  
    TH2 = xr.DataArray(TH,dims=('basin','time'),coords={'basin':basin,'time':years})
    ds = xr.Dataset({'TH':TH2})
    ds.to_netcdf(f'../rawdata/Adyn_temp/{model}_{scen}.nc')
    ds.close()

In [10]:
mods = fun.models()
mods = mods[2:]
for model in mods:
    mask,lon,lat = getgrid(model)
    vol = getvol(model)
    Th0,Th1,Th2,Th3,Th4,firstyear = gethist(model)
    for scen in ['rcp45','rcp85']:
        TH,years = getproj(model,scen,Th0,Th1,Th2,Th3,Th4,firstyear)
        SLRb,years = calcSLR(TH,years)
        SLR = np.sum(SLRb,axis=0)
        save_extra(TH,years,model,scen)
        #Save SLR
        for b in range(0,5):
            fun.saveSLR(SLRb[b,:],years,model,f'{scen}_{b}','Adyn')
            print(np.nanmean(SLRb[b,-20:]))
        fun.saveSLR(SLR,years,model,scen,'Adyn')
        print(np.nanmean(SLR[-20:]))

Added TH 1850
Added TH 1860
Added TH 1870
Added TH 1880
Added TH 1890
Added TH 1900
Added TH 1910
Added TH 1920
Added TH 1930
Added TH 1940
Added TH 1950
Added TH 1960
Added TH 1970
Added TH 1980
Added TH 1990
Added TH 2000
CCSM4 got hist
Added TH 2006
Added TH 2010
Added TH 2020
Added TH 2030
Added TH 2040
Added TH 2050
Added TH 2060
Added TH 2070
Added TH 2080
Added TH 2090
CCSM4 rcp45 got proj
rcp45_0 CCSM4 Saved  Adyn
2.028097408094694
rcp45_1 CCSM4 Saved  Adyn
4.648556546512844
rcp45_2 CCSM4 Saved  Adyn
2.2343118080131896
rcp45_3 CCSM4 Saved  Adyn
3.2489057445597638
rcp45_4 CCSM4 Saved  Adyn
1.2349242909078906
rcp45 CCSM4 Saved  Adyn
13.394795798088381
Added TH 2006
Added TH 2010
Added TH 2020
Added TH 2030
Added TH 2040
Added TH 2050
Added TH 2060
Added TH 2070
Added TH 2080
Added TH 2090
CCSM4 rcp85 got proj
rcp85_0 CCSM4 Saved  Adyn
2.488782232377248
rcp85_1 CCSM4 Saved  Adyn
5.597157905369842
rcp85_2 CCSM4 Saved  Adyn
2.3653330193217856
rcp85_3 CCSM4 Saved  Adyn
3.755343685929